#### Unsupervised Clustering. Expectation-Maximization Clustering (EMC) algorithm.

#### EMC

- unsupervised clustering algorithm
- grups objects into k categories assuming an underlying probability distribution

#### heuristic

- maximize the log-likelihood of the probability distribution parameters

#### algorithm

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
#sns.set_theme()

#### generate synthetic dataset

In [ ]:
from sklearn.datasets import make_classification, make_regression

In [ ]:
X, y = make_classification(n_samples=1000, n_features=5, n_informative = 3, n_classes = 3, n_clusters_per_class = 1, class_sep = 2.0, n_redundant = 0, random_state = 2783)
print(X.shape, y.shape)

#### train/test split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size = 0.2)

#### instantiate Gaussian-mixture model (GMM)

In [ ]:
from sklearn.mixture import GaussianMixture

In [ ]:
gm = GaussianMixture(n_components = 3, random_state = 0).fit(Xtrain)

In [ ]:
gm.means_

In [ ]:
Ypred = gm.predict(Xtest)

#### evaluate GMM

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

In [ ]:
cm = confusion_matrix(Ytest, Ypred)

In [ ]:
#ConfusionMatrixDisplay(cm, display_labels = lbls).plot(ax = axs[0], xticks_rotation = 90.0, values_format = '.2f', cmap = 'GnBu')
ConfusionMatrixDisplay(cm).plot()

#### sample data from the GMM

In [ ]:
gmX = gm.sample(1000)
type(gmX), len(gmX)

In [ ]:
gmX[0].shape, gmX[1].shape

#### GMM data exploration

In [ ]:
df_gm = pd.DataFrame(gmX[0], columns = ['X%d' %j for j in range(gmX[0].shape[1])])
df_gm['target'] = gmX[1]
df_gm.head()

In [ ]:
df_gm[df_gm.columns[:-1]].describe()

In [ ]:
df_gm['target'].value_counts()

In [ ]:
sns.pairplot(df_gm, hue = 'target', corner = True)

#### GMM implementation

In [ ]:
from scipy.stats import multivariate_normal as mvn

In [ ]:
class mygmm():
    
    def __init__(self, k):
        
        self.k = k
        
    def fit(self, Xtrain, maxIterations = 100):
                
        rangeX = Xtrain.max() -Xtrain.min()
        
        # pick random parameters (mean, sdv) for each gaussian component
        # class prior
        self.P = np.random.rand(self.k)
        self.P /= np.sum(self.P)
        # Gaussian means
        self.M = np.array([Xtrain.min() +rangeX *np.random.rand(Xtrain.shape[1]) for k in range(self.k)])
        # Gaussian covariance matrices 
        # - diagonal covariance, independent features
        #self.S = [np.diag(np.diag(np.random.rand(X.shape[1]**2).reshape(X.shape[1], X.shape[1]))) for k in range(self.k)]
        # - full covariance matrix
        self.S = np.random.rand(X.shape[1]**2 *self.k).reshape(self.k, X.shape[1], X.shape[1])
        self.S = [np.sqrt(np.dot(S, S.T)) for S in self.S]

        self.loglkl = np.zeros(maxIterations)
        iteration = 0
        while iteration < maxIterations:
            # E-step
            W = np.zeros(Xtrain.shape[0] *self.k).reshape(Xtrain.shape[0], self.k)
            for i, x in enumerate(Xtrain):
                wi = [p *mvn.pdf(x, mean = m, cov = s) for p, m, s in zip(self.P, self.M, self.S)]
                wi /= np.sum(wi)
                # save weights
                W[i] = wi
            # M-step
            # 1. update prior
            self.P = np.mean(W, axis = 0)
            self.P /= np.sum(self.P)
            # 2. update means
            for c in range(self.k):
                self.M[c] = np.sum(np.dot(W[:, c].T, Xtrain), axis = 0) /np.sum(W[:, c])
            # 3. update covariance matrix
            for c in range(self.k):
                for i in range(Xtrain.shape[1]):
                    for j in range(Xtrain.shape[1]):
                        self.S[c][i,j] = np.sum(W[:, c] *(Xtrain[:, i] -self.M[c][i]) *(Xtrain[:, j] -self.M[c][j])) /np.sum(W[:, c])
            # compute loglikelihood
            self.loglkl[iteration] = np.sum(np.max(W, axis = 0)) /Xtrain.shape[0]
            print('+++ iteration %d %8.6f\r' %(iteration, self.loglkl[iteration]), end = '')
            iteration += 1
                    
    def plot(self):
        plt.plot(self.loglkl)
        plt.show()
        
    def predict(self, Xtest):

        S = -np.ones(Xtest.shape[0])
        # compute densities
        for i, x in enumerate(Xtest):
            wi = [p *mvn.pdf(x, mean = m, cov = s) for p, m, s in zip(self.P, self.M, self.S)]
            wi /= np.sum(wi)
            # assign cluster
            S[i] = np.argmax(wi)
        
        # return predictions
        return S
        

In [ ]:
self = mygmm(3)

In [ ]:
rangeX = Xtrain.max() -Xtrain.min()

# pick random parameters (mean, sdv) for each gaussian component
# class prior
self.P = np.random.rand(self.k)
self.P /= np.sum(self.P)
# Gaussian means
self.M = np.array([Xtrain.min() +rangeX *np.random.rand(Xtrain.shape[1]) for k in range(self.k)])
# Gaussian covariance matrices 
# - diagonal covariance, independent features
#self.S = [np.diag(np.diag(np.random.rand(X.shape[1]**2).reshape(X.shape[1], X.shape[1]))) for k in range(self.k)]
# - full covariance matrix
self.S = np.random.rand(X.shape[1]**2 *self.k).reshape(self.k, X.shape[1], X.shape[1])
self.S = [np.dot(S, S.T) for S in self.S]


In [ ]:
self.S[0]

In [ ]:
x = Xtrain[np.random.randint(Xtrain.shape[0])]
wi = [p *mvn.pdf(x, mean = m, cov = s) for p, m, s in zip(self.P, self.M, self.S)]
wi /np.sum(wi)

In [ ]:
%time _mygmm.fit(Xtrain, maxIterations = 50)

In [ ]:
_mygmm.plot()

In [ ]:
Ypred = _mygmm.predict(Xtest)

In [ ]:
cm = confusion_matrix(Ytest, Ypred)
ConfusionMatrixDisplay(cm).plot()